In [1]:
import requests
import csv

In [2]:
with open("../data/sentences_en.csv", "r") as f:
    dict_reader = csv.DictReader(f, delimiter=",")
    data = list(dict_reader)

In [3]:
def get_wikidata(wikipage):
    response = requests.get(
        url = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&sites=enwiki&titles="+wikipage
        )
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return None
    else:
        response = response.json()
        wbitem = list(response["entities"].keys())[0]
        if wbitem == "-1":
            response = requests.get(
                url = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&sites=enwiki&titles="+wikipage+"&normalize=1"
            )
            response = response.json()
            wbitem = list(response["entities"].keys())[0]
        return wbitem

In [4]:
API_URL = "https://rel.cs.ru.nl/api"
result = requests.post(API_URL, json={
        "text": "Bob Dylan was a singer",
        "spans": []
    }).json()
print(result)

[[0, 9, 'Bob Dylan', 'Bob_Dylan', 0.5031218442860037, 0.9814744591712952, 'PER']]


In [7]:
API_URL = "https://rel.cs.ru.nl/api"
output = []

for item in data:
    text = item["sentence"]
    entities = list() 
    result = requests.post(API_URL, json={
        "text": text,
        "spans": []
    }).json()
    
    if len(result) > 0:
        for entry in result:
            entity = []
            wikipedia_id = entry[3]
            wikidata_id = get_wikidata(wikipedia_id)
            start_pos = entry[0]
            end_pos = entry[1]+entry[0]
            output.append({
                "id":item["id"],
                "start_pos":start_pos,
                "end_pos":end_pos,
                "label":entry[6],
                "surface":text[start_pos:end_pos],
                "alias":wikipedia_id,
                "wb_id":wikidata_id,
                "score":entry[4]
                
            })

In [8]:
a_file = open("../results/rel_en/output_nel.csv", "w")
dict_writer = csv.DictWriter(a_file, keys)
dict_writer.writeheader()
dict_writer.writerows(output)
a_file.close()

In [12]:
text = "And among these models and designs, there was one by which he often demonstrated to many ingenious citizens, who were then governing Florence, how he proposed to raise the Temple of S. Giovanni in Florence, and place steps under it, without damaging the building; and with such strong reasons did he urge this, that it appeared possible, although each man, after he had departed, would recognize for himself the impossibility of so vast an undertaking."
result = requests.post(API_URL, json={
        "text": text,
        "spans": []
    }).json()
    
print(result)

[[133, 8, 'Florence', 'Florence', 0.501479935323753, 0.9240928292274475, 'LOC'], [185, 8, 'Giovanni', 'Pope_Leo_X', 0.1891788090958876, 0.9906159043312073, 'PER'], [197, 8, 'Florence', 'Florence', 0.5312461960888968, 0.9962731599807739, 'LOC']]
